In [2]:
!pip install 'sentence-transformers'
!pip install 'portalocker>=2.0.0'
!pip install 'pytorch-lightning'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=024ee964caf34d8740f4164fae9a0d30396965fb55efdad00a14841277a1f0bf
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 12.9 MB/s eta 0:00:00


In [33]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')  # You can choose a different pre-trained model

# Example sentences
sentences = [
    "Let's train a classifier with BERT.",
    "BERT is a powerful language model."
]

# Get sentence embeddings
sentence_embeddings = model.encode(sentences)

# Print sentence embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {np.round(embedding, 6)}\n")


.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Sentence: Let's train a classifier with BERT.
Embedding: [ 5.534800e-02  7.631680e-01  1.544838e+00  6.226700e-01  6.866600e-02
  1.316600e-02  2.824540e-01 -7.254040e-01  6.869920e-01 -1.470600e-01
 -2.414270e-01  7.999710e-01  4.395990e-01 -3.955300e-02 -9.755070e-01
 -2.550270e-01 -7.964320e-01 -7.310140e-01  9.143100e-01 -1.155684e+00
 -8.146200e-01  1.328360e-01 -1.899010e-01 -8.418400e-02  8.897190e-01
  3.834460e-01 -5.578000e-03 -1.135218e+00 -1.250529e+00  6.421330e-01
 -1.581130e-01  8.905490e-01  3.937370e-01 -4.094340e-01 -1.280968e+00
  2.904270e-01  9.785540e-01  3.524990e-01  4.274270e-01 -2.183610e-01
  1.742770e-01  4.933170e-01  9.046230e-01 -1.699160e-01 -1.114365e+00
 -2.803970e-01 -1.026670e-01 -3.071240e-01  5.611060e-01 -1.780301e+00
 -6.198740e-01  1.108720e-01  3.615600e-01  6.352280e-01 -3.224690e-01
 -1.639380e-01  2.621810e-01 -1.045955e+00 -7.082800e-02 -4.877350e-01
 -8.227040e-01  5.136360e-01  5.006240e-01  6.474620e-01  2.059800e-01
  1.905610e-01  7.44

In [34]:
embedding.shape

(768,)

In [5]:
import torch
from tqdm import tqdm
import torchtext.datasets as datasets
from collections import Counter

In [6]:
train = datasets.IMDB('./', split="train")
test = datasets.IMDB('./', split="test")

In [7]:
train_data = []
for label, text in train:
  train_data.append((label, text))
print("Label Counts for Train Data: ", Counter([_[0] for _ in train_data]))

test_data = []
for label, text in test:
  test_data.append((label, text))
print("Label Counts for Train Data: ", Counter([_[0] for _ in test_data]))

Label Counts for Train Data:  Counter({1: 12500, 2: 12500})
Label Counts for Train Data:  Counter({1: 12500, 2: 12500})


In [36]:
train_sentence_embeddings = model.encode([_[1] for _ in train_data])
test_sentence_embeddings = model.encode([_[1] for _ in test_data])

In [37]:
from sklearn.preprocessing import OneHotEncoder

ohe_enc = OneHotEncoder()
train_label_raw = [[sample[0]] for sample in train_data]
test_label_raw = [[sample[0]] for sample in test_data]

ohe_enc.fit(train_label_raw + test_label_raw)

OneHotEncoder()

In [38]:
train_label = ohe_enc.transform(train_label_raw)
test_label = ohe_enc.transform(test_label_raw)

In [39]:
train_labels_tensor = torch.from_numpy(train_label.todense().astype('float32'))
test_labels_tensor = torch.from_numpy(test_label.todense().astype('float32'))

In [53]:
import numpy as np
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import pytorch_lightning as pl

class Net(pl.LightningModule):

    def __init__(self):
        super(Net, self).__init__()
        self.criterion = torch.nn.BCELoss()
        self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.valid_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.fc1 = nn.Linear(768, 120)  # input dimension with 2000 words/tokens
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        accuracy = self.train_acc(logits.round(), y)
        self.log('train_loss', loss)
        self.log('train_acc', accuracy, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        accuracy = self.valid_acc(logits.round(), y)
        self.log('val_loss', loss)
        self.log('val_acc', accuracy, on_epoch=True)

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt

net = Net()
print(net)


Net(
  (criterion): BCELoss()
  (train_acc): MulticlassAccuracy()
  (valid_acc): MulticlassAccuracy()
  (fc1): Linear(in_features=768, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [41]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

train_dataset = CustomDataset(train_sentence_embeddings, train_labels_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = CustomDataset(test_sentence_embeddings, test_labels_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [42]:
sample = next(iter(train_dataloader))

In [43]:
sample[0].shape, sample[1].shape

(torch.Size([128, 768]), torch.Size([128, 2]))

In [56]:
trainer = pl.Trainer(max_epochs=20)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [57]:
trainer.validate(net, test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.4983200132846832     │
│         val_loss          │    0.6932352185249329     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.6932352185249329, 'val_acc': 0.4983200132846832}]

In [58]:
trainer.fit(net, train_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | criterion | BCELoss            | 0     
1 | train_acc | MulticlassAccuracy | 0     
2 | valid_acc | MulticlassAccuracy | 0     
3 | fc1       | Linear             | 92.3 K
4 | fc2       | Linear             | 10.2 K
5 | fc3       | Linear             | 170   
-------------------------------------------------
102 K     Trainable params
0         Non-trainable params
102 K     Total params
0.410     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [59]:
trainer.validate(net, test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8178399801254272     │
│         val_loss          │    0.4495897889137268     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.4495897889137268, 'val_acc': 0.8178399801254272}]